In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from collections import OrderedDict
import numpy as np
from torchvision.models import vgg16
from codecarbon import EmissionsTracker


tracker = EmissionsTracker()
tracker.start()



def get_vgg16_model(num_classes):
    model = vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, num_classes)
    return model



def train_model(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()

def test_model(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss(reduction='sum')(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    return test_loss / len(test_loader.dataset), correct / len(test_loader.dataset)

def average_weights(weight_list):
    avg_weights = OrderedDict()
    for key in weight_list[0].keys():
        avg_weights[key] = sum([client_weights[key] for client_weights in weight_list]) / len(weight_list)
    return avg_weights
    
def create_client_dataset(dataset, num_clients):
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)
    client_indices = np.array_split(indices, num_clients)
    client_datasets = [Subset(dataset, index_list) for index_list in client_indices]
    return client_datasets

# Load the CIFAR-10 dataset
transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=4), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

num_clients = 5
client_datasets = create_client_dataset(train_dataset, num_clients)
train_loaders = [DataLoader(client_dataset, batch_size=100, shuffle=True) for client_dataset in client_datasets]
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 10


# ... (previous code remains the same) ...

def federated_learning(train_loaders, test_loader, device, epochs, num_clients, communication_rounds, client_weighting, clients_per_round):
    global_model = get_vgg16_model(10).to(device)
    client_models = [get_vgg16_model(10).to(device) for _ in range(num_clients)]

    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())

    for com_round in range(1, communication_rounds + 1):
        print(f'Communication round {com_round}/{communication_rounds}')

        selected_clients = np.random.choice(range(num_clients), size=clients_per_round, replace=False)
        client_weights = []

        client_accuracies = []
        if client_weighting:
            for idx in selected_clients:
                client_model = client_models[idx]
                test_loss, client_accuracy = test_model(client_model, device, test_loader)
                client_accuracies.append(client_accuracy)
            max_accuracy = max(client_accuracies)

        for idx, client_idx in enumerate(selected_clients):
            client_model = client_models[client_idx]
            optimizer = optim.SGD(client_model.parameters(), lr=0.001, momentum=0.9)
            train_model(client_model, device, train_loaders[client_idx], optimizer, epochs)

            if client_weighting:
                client_weight = client_accuracies[idx] / max_accuracy
                print(client_weight)
            else:
                client_weight = 1

            weighted_client_state = OrderedDict()

            for key in client_model.state_dict().keys():
                weighted_client_state[key] = client_model.state_dict()[key] * client_weight

            client_weights.append(weighted_client_state)

        global_weights = average_weights(client_weights)
        global_model.load_state_dict(global_weights)

        test_loss, accuracy = test_model(global_model, device, test_loader)
        print(f'Test loss: {test_loss:.4f}, Accuracy: {accuracy * 100:.2f}%\n')

    return global_model



communication_rounds = 50
client_weighting = True
clients_per_round = 5

federated_model = federated_learning(train_loaders, test_loader, device, epochs, num_clients, communication_rounds, client_weighting, clients_per_round)


tracker.stop()


/home/gaia/anaconda3/envs/jessVIT/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[codecarbon INFO @ 14:03:42] [setup] RAM Tracking...
[codecarbon INFO @ 14:03:42] [setup] GPU Tracking...
[codecarbon INFO @ 14:03:42] No GPU found.
[codecarbon INFO @ 14:03:42] [setup] CPU Tracking...
[codecarbon ERROR @ 14:03:42] Unable to read Intel RAPL files for CPU power, we will use a constant for your CPU power. Please view https://github.com/mlco2/codecarbon/issues/244 for workarounds : [Errno 13] Permission denied: '/sys/class/powercap/intel-rapl/intel-rapl:1/energy_uj'
[codecarbon ERROR @ 14:03:42] Unable to read Intel RAPL files for CPU power, we will use a constant for your CPU power. Please view https://github.com/mlco2/codecarbon/issues/244 for workarounds : [Errno 13] Permission denied: '/sys/class/powercap/

Files already downloaded and verified
Files already downloaded and verified


/home/gaia/anaconda3/envs/jessVIT/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/gaia/anaconda3/envs/jessVIT/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Communication round 1/50


[codecarbon INFO @ 14:04:02] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 14:04:02] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 14:04:02] 0.000024 kWh of electricity used since the begining.
[codecarbon INFO @ 14:04:17] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 14:04:17] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 14:04:17] 0.000048 kWh of electricity used since the begining.
[codecarbon INFO @ 14:04:32] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 14:04:32] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 14:04:32] 0.000072 kWh of electricity used since the begining.
[codecarbon INFO @ 14:04:47] Energy consumed for RAM : 0.000096 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 14:04:47] Energy consumed for all CPUs :

...

[codecarbon INFO @ 15:01:17] Energy consumed for RAM : 0.005515 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 15:01:17] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 15:01:17] 0.005515 kWh of electricity used since the begining.
[codecarbon INFO @ 15:01:32] Energy consumed for RAM : 0.005539 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 15:01:32] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 15:01:32] 0.005539 kWh of electricity used since the begining.
[codecarbon INFO @ 15:01:47] Energy consumed for RAM : 0.005563 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 15:01:47] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 15:01:47] 0.005563 kWh of electricity used since the begining.


1.0


[codecarbon INFO @ 15:02:02] Energy consumed for RAM : 0.005587 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 15:02:02] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 15:02:02] 0.005587 kWh of electricity used since the begining.
[codecarbon INFO @ 15:02:17] Energy consumed for RAM : 0.005611 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 15:02:17] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 15:02:17] 0.005611 kWh of electricity used since the begining.
[codecarbon INFO @ 15:02:32] Energy consumed for RAM : 0.005635 kWh. RAM Power : 5.755537033081055 W
[codecarbon INFO @ 15:02:32] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 15:02:32] 0.005635 kWh of electricity used since the begining.


: 

: 